Este projeto é uma análise exploratória do dataset Intel Image Classification, disponível no Kaggle. O dataset contém 25 mil imagens de 6 classes diferentes, sendo elas: buildings, forest, glacier, mountain, sea e street. O objetivo é classificar as imagens de acordo com a classe a qual pertencem.



In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

2023-12-05 15:05:43.928135: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-05 15:05:44.028338: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 15:05:44.028413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 15:05:44.032897: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 15:05:44.048197: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-05 15:05:44.050739: I tensorflow/core/platform/cpu_feature_guard.cc:1

### Preparing Dataset

In [2]:
train_dataset = keras.utils.image_dataset_from_directory('./seg_train', batch_size=64, image_size=(150, 150),  label_mode='categorical')
validation_dataset = keras.utils.image_dataset_from_directory('./seg_test', batch_size=64, image_size=(150, 150),  label_mode='categorical')
prediction_dataset = keras.utils.image_dataset_from_directory('./seg_pred', batch_size=64,  shuffle=True, image_size=(150, 150))

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.
Found 7301 files belonging to 1 classes.


## Exploratore Analysis

Como veremos abaixo, o dataset é composto por imagens de 150x150 pixels, coloridas e com 3 canais (RGB). Além disso, o dataset é balanceado, ou seja, possui a mesma quantidade de imagens para cada classe.
Exibiremos algumas imagens do dataset para melhor entendimento.



In [3]:
# # Describe some characteristics of the dataset.
# print(train_dataset.class_names)
# print(validation_dataset.class_names)
# print(train_dataset)
# print(validation_dataset)

# # Visualize one of the images in the dataset.
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_dataset.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(train_dataset.class_names[labels[i]])
#         plt.axis("off")

### Model Definition

In [4]:
neural_networks = [
    {
        'description': 'Conv2D_MaxPooling_Conv2D',
        'model': Sequential([
                    Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
                    MaxPooling2D(pool_size=(2, 2)),
                    Convolution2D(filters=32, kernel_size=(3, 3), activation='relu'),
                    Flatten(input_shape=(150, 150, 3)),
                    Dense(units=6, activation='softmax')
                    ]
                ),
        'epochs': 2,
        # 'steps_per_epoch': 100,
        # 'validation_steps': 10,
        'optimizer': Adam(learning_rate=0.0001),
        'loss_function': 'categorical_crossentropy',
        'accuracy': None,
        'loss': None,
        'training_time': None,
    }
]

In [5]:
for nn in neural_networks:
    model = nn['model']
    print(nn)
    model.compile(optimizer=nn['optimizer'], loss=nn['loss_function'], metrics=['accuracy'])

{'description': 'Conv2D_MaxPooling_Conv2D', 'model': <keras.src.engine.sequential.Sequential object at 0x7f10a29b81c0>, 'epochs': 2, 'optimizer': <keras.src.optimizers.adam.Adam object at 0x7f10a29b8130>, 'loss_function': 'categorical_crossentropy', 'accuracy': None, 'loss': None, 'training_time': None}


In [6]:
for nn in neural_networks:
    model = nn['model']
    model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=nn['epochs'],
        verbose=2
    )
    model.save('models/' + nn['description'] + '.h5')
    model.evaluate(prediction_dataset, verbose=2)


Epoch 1/2


220/220 - 268s - loss: 22.3919 - accuracy: 0.4963 - val_loss: 5.2914 - val_accuracy: 0.5647 - 268s/epoch - 1s/step
Epoch 2/2
220/220 - 214s - loss: 2.0736 - accuracy: 0.6994 - val_loss: 2.6276 - val_accuracy: 0.6440 - 214s/epoch - 971ms/step


/home/gxara/.pyenv/versions/3.9.0/envs/ml/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


ValueError: in user code:

    File "/home/gxara/.pyenv/versions/3.9.0/envs/ml/lib/python3.9/site-packages/keras/src/engine/training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "/home/gxara/.pyenv/versions/3.9.0/envs/ml/lib/python3.9/site-packages/keras/src/engine/training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/gxara/.pyenv/versions/3.9.0/envs/ml/lib/python3.9/site-packages/keras/src/engine/training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "/home/gxara/.pyenv/versions/3.9.0/envs/ml/lib/python3.9/site-packages/keras/src/engine/training.py", line 1919, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/gxara/.pyenv/versions/3.9.0/envs/ml/lib/python3.9/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/home/gxara/.pyenv/v

## Prediction

In [ ]:
# predictions = model.predict(prediction_dataset, verbose=2)

## Visualizing

In [ ]:
# from sklearn.metrics import confusion_matrix

# y_pred = np.argmax(predictions, axis=1)
# y_true = np.concatenate([y for x, y in prediction_dataset], axis=0)
# cm = confusion_matrix(y_true, y_pred)